# PROGRAM 16-2: Julia Sets
Iterate complex functions of the form $f(z) = z^{2} + c$.  
$c$ is a chosen constant complex number.  
Form a complex value $z = x + yi$ for each point the $xy$ plane.  
If the modulus of any $z$ orbit value is greater than ESCAPE_MODULUS,
then the orbit has escaped. If there is no escape after MAX_ITERS number
of iterations, then the orbit is bounded.  
For each value of $z$ whose orbit escapes, plot a point in  a shade of
grey to represent the point's escape velocity. The more iterations it
took to escape, the darker the shade.  
If the orbit did not escape, plot a point using a color based on the
modulus of the last computed value


In [1]:
import numpy as np
import random
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import TextInput, Button, Select, Div
from bokeh.layouts import layout, column
output_notebook()


Loading BokehJS ...

In [2]:
MAX_ITERS = 32
ESCAPE_MODULUS = 2
MAX_ALPHA = 255
PRESETS = {"": 0 + 0j, 
           "Claws": 0.30900264 - 0.0339787j,
           "Snowflakes": 0.33843684 - 0.4211402j, 
           "Turtles": -0.3985014 + 0.5848901j,
           "Serpents": -0.8184639 - 0.2129812j, 
           "Amoebas": -0.3346212 + 0.6340579j, 
           "Sparklers": -0.5530404 + 0.5933997j}

def julia_set(c, width, height, x_min, x_max, y_min, y_max):
    img = np.empty((height, width), dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((height, width, 4))
    x_delta = (x_max - x_min) / width
    y_delta = (y_max - y_min) / height
    for row in range(height):
        y0 = y_min + row * y_delta
        for col in range(width):
            x0 = x_min + col * x_delta
            z = x0 + y0 * 1j  # z = x0 + y0i
            r, g, b = rgb(z, c)
            view[row, col, 0] = r
            view[row, col, 1] = g
            view[row, col, 2] = b
            view[row, col, 3] = MAX_ALPHA
    return img

def rgb(z, c):
    escaped = False
    iters = 0
    modulus = 0
    
    while iters < MAX_ITERS and not escaped:
        z = z * z + c
        modulus = abs(z)  # abs() calculates modulus
        escaped = modulus > ESCAPE_MODULUS
        iters += 1
    
    if escaped:
        k = 255 - (255 * iters) / MAX_ITERS
        k = min(k, 240)
        return k, k, k
    else:
        m = int(int(100 * modulus) / ESCAPE_MODULUS + 1)
        r = (307 * m) & 255
        g = (353 * m) & 255
        b = (401 * m) & 255
        return r, g, b


In [3]:
def modify_doc(doc):
    real_in = TextInput(value="0", title="Real:")
    imag_in = TextInput(value="0", title="Imaginary:")
    sel = Select(title="Presets", value="", options=list(PRESETS.keys()))
    div = Div(text='''<b>Julia Set of z^2 + </b><i>Real</i><b> + </b>
        <i>Imaginary</i><b>i</b>''')
    refresh_but = Button(label="Refresh", button_type="success")
    random_but = Button(label="Random", button_type="success")
    
    p = figure(x_range=(-2, 2), y_range=(-2,2), 
               tooltips=[("x", "$x"), ("y", "$y")])
    source = ColumnDataSource(data=dict(image=[], x=[], y=[], dw=[], dh=[]))
    p.image_rgba(image='image', x='x', y='y', dw='dw', dh='dh', source=source)
    
    def refresh():
        x_min = p.x_range.start
        x_max = p.x_range.end
        y_min = p.y_range.start
        y_max = p.y_range.end
        source.data = dict(
            image=[julia_set(
                float(real_in.value) + float(imag_in.value) * 1j,
                p.inner_width, p.inner_height,
                x_min, x_max,
                y_min, y_max)],
            x=[x_min],
            y=[y_min],
            dw=[x_max - x_min],
            dh=[y_max - y_min]
        )
    refresh_but.on_click(refresh)
    
    def handle_select(attr, old, new):
        comp = PRESETS[sel.value]
        real_in.value = str(comp.real)
        imag_in.value = str(comp.imag)
        refresh()
    sel.on_change('value', handle_select)
    
    def handle_rand():
        real = random.uniform(-1, 1)
        imag = random.uniform(-1, 1)
        real_in.value = str(real)
        imag_in.value = str(imag)
        refresh()
    random_but.on_click(handle_rand)
    
    widget_col1 = column(real_in, imag_in, sel)
    widget_col2 = column(div, refresh_but, random_but)
    l = layout([p],[widget_col1, widget_col2])
    doc.add_root(l)

show(modify_doc)
